In [1]:
import itertools
import numpy as np
from numpy import linalg as LA
import math

In [117]:
# number of sites
N = 6

def generate_basis_states(n):
    """
    Generates basis states for half filling of N sites
    """
    result = []
    int_result = []
    for bits in itertools.combinations(range(n * 2), n):
        # only choosing N bits from N * 2 bits, so half filling
        s = ['0'] * n * 2
        for bit in bits:
            # first n bits are up, second n are down
            # sites go 0 to N - 1 for each spin
            s[bit] = '1'
        
        if (''.join(s)[:N].count('1') == N / 2): # half filling
            int_result.append(int(''.join(s),2)) # integer representation of bit basis
            result.append(''.join(s))            # bit (string) basis
    # order ascending
    return (result[::-1], int_result[::-1])

(result, basis_states) = generate_basis_states(N)

In [118]:
# specify interaction and hopping terms
model = [
    {"U": 2, "neighbors": [1,3], "hoppings": [0,1,0,1]},
    {"U": 2, "neighbors": [0,2], "hoppings": [1,0,1,0]},
    {"U": 2, "neighbors": [1,3], "hoppings": [0,1,0,1]},
    {"U": 2, "neighbors": [0,2], "hoppings": [1,0,1,0]}
]

model2 = [
    {"U": 2, "neighbors": [1], "hoppings": [0,1]},
    {"U": 2, "neighbors": [0], "hoppings": [1,0]}
]

model = [
    {"U": 2, "neighbors": [1,2,5], "hoppings": [0,1,1,0,0,1]},
    {"U": 2, "neighbors": [0,2,4], "hoppings": [1,0,1,0,1,0]},
    {"U": 2, "neighbors": [0,1,3], "hoppings": [1,1,0,1,0,0]},
    {"U": 2, "neighbors": [2,4,5], "hoppings": [0,0,1,0,1,1]},
    {"U": 2, "neighbors": [1,3,5], "hoppings": [0,1,0,1,0,1]},
    {"U": 2, "neighbors": [1,3,4], "hoppings": [1,0,0,1,1,0]}
]

# 0 up; 1 up; 2 up; 3 up; ... ||  0 down; 1 down; 2 down; 3 down; ...

In [119]:
# setting up U term in half-filling space
h_u = np.array([[ 0 for i in range(len(result))] for j in range(len(result))])

#          up     down
spins = [(x[:N], x[N:]) for x in result]

for i in range(len(spins)):
    # if two particles are on the same site this is recorded in h_u. Only diagonal is affected
    # i.e. ('00', '11') --> 0*1 + 0*1 = 0
    #      ('01', '01') --> 0*0 + 1*1 = 1  both particles are on side 1
    sites = []
    for j in range(N):
        if (int(spins[i][0][j]) * int(spins[i][1][j]) == 1):
            sites.append(j)
    if (len(sites) > 0):
        h_u[i][i] = sum([model[x]["U"] for x in sites])

In [120]:
# setting up t term in half-filling space
h_t = np.array([[ 0 for i in range(len(result))] for j in range(len(result))])

def hops(state):
    def ns(s):
        """
        Finds the possible neighbor states for a state s. Spins can't flip while hopping
        """
        hops = []
        for i in range(len(s)):
            if (s[i] == '1'):
                # check all neighbors of model[i]
                for j in model[i]["neighbors"]:
                    if s[j] == '0':
                        # count the number of 1s between i and j
                        c = s[i+1:j].count('1') if i < j else s[j+1:i].count('1')
                        # convert to floats to get rid of -0 bug
                        hops.append((float(i), (-1)**c * float(j)))
        return hops

    def simulate_hop(s, hop):
        """
        Returns a new state in the bit basis after hopping
        """
        s = list(s)
        s[abs(int(hop[0]))] = '0'
        s[abs(int(hop[1]))] = '1'
        return "".join(s)

    
    neighbor_states = []
    # find possible hops for particles with up spin
    up_hops = ns(state[0])
    # find possible hops for particles with down spin
    down_hops = ns(state[1])

    start = basis_states.index(int(state[0] + state[1], 2))
    for i in range(len(down_hops)):
        hop = state[0] + simulate_hop(state[1], down_hops[i])
        hop_index = basis_states.index(int(hop, 2))
        if (math.copysign(1, down_hops[i][1]) == -1.0):
            # model[down_hops[i][0]] selects starting site
            # hops to down_hops[i][1], use model array to get specific t value for the hop
            # entry in h_t is positive because we multiply by negative t
            h_t[start][hop_index] = model[int(down_hops[i][0])]["hoppings"][int(down_hops[i][1])]
        else:
            # entry in h_t is negative because we multiply by negative t
            h_t[start][hop_index] = -model[int(down_hops[i][0])]["hoppings"][int(down_hops[i][1])]


    for i in range(len(up_hops)):
        hop = simulate_hop(state[0], up_hops[i]) + state[1]
        hop_index = basis_states.index(int(hop, 2))
        if (math.copysign(1, up_hops[i][1]) == -1.0):
            h_t[start][hop_index] = model[int(up_hops[i][0])]["hoppings"][int(up_hops[i][1])]
        else:
            h_t[start][hop_index] = -model[int(up_hops[i][0])]["hoppings"][int(up_hops[i][1])]

for i in range(len(spins)):
    hops(spins[i])

In [121]:
h = h_u + h_t
#np.savetxt('h.txt', h, '%-2d', ', ')
print(h)
print(np.count_nonzero(h))
print(len(h[0]))

[[ 6 -1  0 ...  0  0  0]
 [-1  4 -1 ...  0  0  0]
 [ 0 -1  4 ...  0  0  0]
 ...
 [ 0  0  0 ...  4 -1  0]
 [ 0  0  0 ... -1  4 -1]
 [ 0  0  0 ...  0 -1  6]]
4220
400


In [122]:
w, v = LA.eigh(h)
print(w)

[-6.14893707 -5.21796009 -5.02749009 -4.76148729 -4.56721594 -4.53427456
 -4.20477744 -4.14822619 -3.95315474 -3.77144677 -3.51182368 -3.5043079
 -3.25950311 -3.21572491 -2.96938419 -2.96229835 -2.90524973 -2.81985138
 -2.76891915 -2.74251181 -2.65673627 -2.62431753 -2.6052141  -2.58111642
 -2.50013635 -2.45842944 -2.20982257 -2.14764575 -2.09441122 -2.07675461
 -1.99742891 -1.96974783 -1.91713915 -1.78084433 -1.76588658 -1.7443272
 -1.60721201 -1.59267558 -1.56151761 -1.52318805 -1.39925294 -1.36992778
 -1.35560116 -1.28372328 -1.24432306 -1.24293369 -1.19928433 -1.17378181
 -1.14995503 -1.05748861 -1.05683914 -1.05627051 -1.03383385 -1.02852626
 -1.02555376 -0.81615334 -0.71637934 -0.70941708 -0.70892198 -0.54280533
 -0.53982862 -0.52770584 -0.52130214 -0.47198672 -0.43852843 -0.40025124
 -0.37970319 -0.35040964 -0.31209208 -0.30918991 -0.29385288 -0.29259984
 -0.27980037 -0.24408689 -0.12351113 -0.1201861  -0.06936241 -0.05574955
 -0.03931218 -0.03916658 -0.03879072  0.05709458  0.0

In [108]:
def creation(state, site, spin):
    """
    Creates a particle at site 'site' with spin 'spin', if possible
    |0011> has sites 0, 1, in that order per N sites
    param spin: 1 - up, 0 - down
    """
    state = list(state)
    #             up                    down
    particle = state[:N] if spin else state[N:]
    if (particle[site] == '1'):
        # Can't add a particle to an already filled site, return vacuum state
        return ''
    else:
        particle[site] = '1'
        if (spin):
            state[:N] = particle
        else:
            state[N:] = particle
        return state

def annihilation(state, site, spin):
    """
    Annihilates a particle at site 'site' with spin 'spin', if possible
    |0011> has sites 0, 1, in that order per N sites
    param spin: 1 - up, 0 - down
    """
    state = list(state)
    #             up                    down
    particle = state[:N] if spin else state[N:]

    if (particle[site] == '0'):
        # State has been destroyed, so return vacuum state
        return ''
    else:
        particle[site] = '0'
        if (spin):
            state[:N] = particle
        else:
            state[N:] = particle
        return state


In [109]:
# Building the single particle density matrix
# 1 for up, 0 for down
# sigma1, sigma2 up to N. i, j sites

spdm = {}

for sigma1 in range(2):
    for i in range(N):        
        for sigma2 in range(2):
            for j in range(N):
                # setting up M(i, j, sigma1, sigma2)
                arr = np.array([[ 0 for i in range(len(result))] for j in range(len(result))])
                
                for mu in range(len(result)):
                    for nu in range(len(result)):
                        # < nu | a_{i, sigma1}^dagger a_{j, sigma2} | mu >
                
                        phi1 = annihilation(result[nu], i, sigma1)
                        phi2 = annihilation(result[mu], j, sigma2)
                        if ((phi1 != '' and phi2 != '') and phi1 == phi2):
                            arr[mu][nu] = 1
                spdm[str(i)+str(sigma1)+str(j)+str(sigma2)] = arr

In [110]:
almost_spdm_arr = np.array([0.0 for i in range(len(spdm))])
for i, j in enumerate(spdm.values()):
    almost_spdm_arr[i] = np.dot(v[:, 0].conj().T, np.dot(j, v[:, 0]))
asa = np.reshape(almost_spdm_arr, (2*N,2*N))
print(np.around(asa, decimals=2))
print(np.trace(asa))

[[0.5  0.45 0.   0.  ]
 [0.45 0.5  0.   0.  ]
 [0.   0.   0.5  0.45]
 [0.   0.   0.45 0.5 ]]
1.9999999999999991


In [13]:
def matrix_from_pauli(s):
    def tuple_from_pauli(s, tup):
        if (s[0] == 'X' or s[0] == 'Y'):
            new_tup = []
            for i in tup:
                if (i[1][int(s[1])] == '0'):
                    new_str = i[1][:int(s[1])] + '1' + i[1][int(s[1])+1:]
                    new_tup.append([i[0], new_str])
                else:
                    new_str = i[1][:int(s[1])] + '0' + i[1][int(s[1])+1:]
                    new_tup.append([i[0], new_str])
            return new_tup
        else:
            new_tup = []
            for i in tup:
                if (i[1][int(s[1])] == '1'):
                    new_str = i[1][:int(s[1])] + '-1' + i[1][int(s[1])+1:]
                    new_tup.append([i[0], new_str])
                elif (i[1][int(s[1])] == '-1'):
                    new_str = i[1][:int(s[1])] + '1' + i[1][int(s[1])+1:]
                    new_tup.append([i[0], new_str])
                else:
                    new_tup.append([i[0], i[1]])
            return new_tup

    original = [[x, y] for x, y in enumerate(result)]
    tup = original
    for i in range(len(s)-2, -1, -2):
        tup = tuple_from_pauli(s[i:i+2], tup)


    # need to fix the Z's (negatives) here
    for i in range(len(result)):
        num_negatives = tup[i][1].count('-')
        tup[i][1] = tup[i][1].replace('-', '')
        tup[i].append(num_negatives)

    # if hopping brings it out of half filling, revert to original
    for i in range(len(result)):
        if (tup[i][1] not in result):
            tup[i] = original[i]
            tup[i].append(0)

    def sort_by_bitstring(e):
        return int(e[1], 2)
    tup.sort(key=sort_by_bitstring)

    arr = np.array([[ 0 for i in range(len(result))] for j in range(len(result))])
    for i in range(len(result)):
        arr[i][tup[i][0]] = (-1)**(tup[i][2])
    return arr

In [14]:
"""
m = np.eye(36) - matrix_from_pauli('Z0') - matrix_from_pauli('Z4') + matrix_from_pauli('Z0Z4')
m = m + np.eye(36) - matrix_from_pauli('Z1') - matrix_from_pauli('Z5') + matrix_from_pauli('Z1Z5')
m = m + np.eye(36) - matrix_from_pauli('Z2') - matrix_from_pauli('Z6') + matrix_from_pauli('Z2Z6')
m = m + np.eye(36) - matrix_from_pauli('Z3') - matrix_from_pauli('Z7') + matrix_from_pauli('Z3Z7')
m = m / 4


np.savetxt('m.txt', m, '%-2d', ', ')

mt = matrix_from_pauli('X0X1') + matrix_from_pauli('Y0Y1') + matrix_from_pauli('X0Z1Z2X3') + matrix_from_pauli('Y0Z1Z2Y3')
mt = mt + matrix_from_pauli('X1X2') + matrix_from_pauli('Y1Y1') + matrix_from_pauli('X2X3') + matrix_from_pauli('Y2Y3')
mt = mt + matrix_from_pauli('X4X5') + matrix_from_pauli('Y4Y5') + matrix_from_pauli('X4Z5Z6X7') + matrix_from_pauli('Y4Z5Z6Y7')
mt = mt + matrix_from_pauli('X5X6') + matrix_from_pauli('Y5Y6') + matrix_from_pauli('X6X7') + matrix_from_pauli('Y6Y7')
mt = mt / -2


np.savetxt('mt.txt', mt, '%-2d', ', ')
"""
matrix_from_pauli('X0X1') + matrix_from_pauli('Y0Y1') + matrix_from_pauli('X2X3') + matrix_from_pauli('Y2Y3')

array([[4, 0, 0, ..., 0, 0, 0],
       [0, 4, 0, ..., 0, 0, 0],
       [0, 0, 4, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 4, 0, 0],
       [0, 0, 0, ..., 0, 4, 0],
       [0, 0, 0, ..., 0, 0, 4]])